<h1>  Importing Libraries  </h1>


In [1]:
import pandas as pd
import pandasql as ps
import psycopg2
import numpy as np
import json
from datetime import datetime
from datetime import timedelta



<h1>  Loading JSON file  </h1>


In [2]:
with open("/home/ys/notebook/config.json") as f:
    config = json.load(f)
print("Config:",config)
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')


Config: {'database': 'Students', 'user': 'postgres', 'password': 'a', 'host': 'localhost', 'port': '5432'}


<h1>  Time Table  </h1>


In [3]:
mycursor = mydb.cursor()
table="time"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'Students_db' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
     print(value[3],":",value[7])

id : integer
day : date


<h1>  Adding days to Time table  </h1>


In [4]:
def add_day(day):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
   
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT * FROM "Students_db".time where day ='{day}';""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = """INSERT INTO "Students_db".time (day) VALUES ( ' """+day+ """')"""
        val = day
        mycursor.execute(sql)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"""SELECT * FROM "Students_db".time where day ='{day}';""")
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [5]:
for m in range (1,6) :
    df = pd.read_csv("/home/ys/notebook/processed/meetings_processed/meeting_("+str(m)+")_list_processed.csv")
    day=df["date"][1]

    id_day=add_day(day)
    print(id_day," : ",day)

1  :  2021/03/30
2  :  2021/04/06
3  :  2021/04/14
4  :  2021/04/20
5  :  2021/05/05


<h1>  Subject table  </h1>


In [6]:
mycursor = mydb.cursor()
table="subject"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'Students_db' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
     print(value[3],":",value[7])

id : integer
subname : character varying
id_prof : integer


<h1> Adding to Subject table     </h1>


In [7]:
def add_subject(subject,id_prof):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT * FROM "Students_db".subject where subname ='{subject}' and id_prof = '{id_prof}';""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = """INSERT INTO "Students_db".subject (subname,id_prof) VALUES  ('"""+subject+"""','"""+str(id_prof)+"""')"""
        val = (subject,id_prof)
        mycursor.execute(sql)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"""SELECT * FROM "Students_db".subject where subname ='{subject}';""")
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [8]:
def get_id_prof(email):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT id FROM "Students_db".professor where email ='{email}' ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]


In [9]:
for m in range (1,6) :
    df = pd.read_csv("/home/ys/notebook/processed/meetings_processed/meeting_("+str(m)+")_list_processed.csv")

    email = ps.sqldf("select email from df where role = 'Organisateur'", locals())["email"][0]
    subject = df["subject"][1]

    id_prof = get_id_prof(email)
    id_sub = add_subject(subject,id_prof)

    print('subject : ',id_sub," subname : ",subject,' id prof',id_prof)

subject :  1  subname :  Data Warehouse  id prof 1
subject :  1  subname :  Data Warehouse  id prof 1
subject :  1  subname :  Data Warehouse  id prof 1
subject :  1  subname :  Data Warehouse  id prof 1
subject :  1  subname :  Data Warehouse  id prof 1


<h1> Making Data fit our needs   </h1>


In [10]:
def get_id_student(email):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT id FROM "Students_db".student where email ='{email}' ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    
def get_id_time(day):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT id FROM "Students_db".time where day ='{day}' ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    
def get_id_subject(email,subname):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    id_prof = get_id_prof(email)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT id FROM "Students_db".subject where subname ='{subname}' and id_prof = '{str(id_prof)}' ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]

def get_all_student_emails():
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    df_email =  pd.DataFrame(columns = ["email"] )
    email = []
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT email FROM "Students_db".student ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        for j in range(len(myresult)) :
            
            email.append((myresult[j][0]))
        #print(email)           
        df_email ["email"] = email
        
        return df_email

In [11]:
header_df = ['id_student',"id_time",'id_subject','arrive_time','duration',"is_abs","is_late"]


In [12]:
for m in range (1,6) :
    df = pd.read_csv("/home/ys/notebook/processed/meetings_processed/meeting_("+str(m)+")_list_processed.csv")

    df_void = pd.DataFrame(columns = header_df )

    df_student = ps.sqldf("select * from df where role = 'Presentateur' ", locals())
    df_prof = ps.sqldf("select * from df where role = 'Organisateur' ", locals())


    day_id = get_id_time(df_prof["date"][0])


    start =  datetime.strptime(str(df_prof["arrive_time"][0]), '%H:%M:%S')


    df_emails = ps.sqldf("select DISTINCT email from df where role = 'Presentateur'", locals())

    sub_id = get_id_subject(df_prof["email"][0],df_student['subject'][1]) 

    art = []
    duration = []
    id_student = []
    id_time = []
    id_subject = []
    is_abs = []
    is_late = []
    for index, row in df_emails.iterrows():
        arrive_time = []
        duration_max =  datetime.strptime('00:00:00', '%H:%M:%S')
        student_id = get_id_student(row["email"])
        for index_1, row_1 in df_student.iterrows():
            if row["email"] == row_1["email"] :

                a =  datetime.strptime(row_1['duration'] , '%H:%M:%S')
                duration_max +=  timedelta(hours =a.hour, minutes = a.minute, seconds = a.second)

                arrive_time.append(row_1['arrive_time'])

        arrive_time = arrive_time[0]

        delta = datetime.strptime(str(arrive_time), '%H:%M:%S')  - start
        delta = datetime.strptime(str(delta), '%H:%M:%S')
        if   duration_max <  datetime.strptime('00:30:00', '%H:%M:%S') : 
            is_abs.append(True)
            is_late.append(False)
        elif delta   > datetime.strptime('00:10:00', '%H:%M:%S') : 
            is_abs.append(False)
            is_late.append(True)
        else : 
            is_abs.append(False)
            is_late.append(False)


        art.append(arrive_time)
        duration.append(str(duration_max).split(' ')[1])
        id_student.append( student_id )
        id_time.append( day_id )
        id_subject.append( sub_id )


    df_all_eamils = get_all_student_emails() 


    df_emails_abs = ps.sqldf("select email from df_all_eamils where email not in (select email from df_emails)", locals())

    for index, row in df_emails_abs.iterrows():
        student_id = get_id_student(row["email"])

        art.append('00:00:00')
        duration.append('00:00:00')
        id_student.append( student_id )
        id_time.append( day_id )
        id_subject.append( sub_id )
        is_abs.append(True)
        is_late.append(False)

    df_void['id_student'] = id_student
    df_void['id_time'] = id_time
    df_void['id_subject'] = id_subject
    df_void['arrive_time'] = art
    df_void['duration'] = duration
    df_void['is_abs'] = is_abs
    df_void['is_late'] = is_late

    output = "/home/ys/notebook/processed/meetings_output/meeting_("+str(m)+")_list_output.csv"
    df_void.to_csv(output,index=False)
    print("Done :"+str(m))

print("Done Total")



Done :1
Done :2
Done :3
Done :4
Done :5
Done Total


<h1>   Student_abs Table </h1>


In [13]:
mycursor = mydb.cursor()
table="student_abs"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'Students_db' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
     print(value[3],":",value[7])

id : integer
id_student : integer
id_subject : integer
id_time : integer
arrive_time : time without time zone
duration : time without time zone
is_late : boolean
is_abs : boolean


<h1>   Adding to Student_abs Table </h1>


In [14]:

    
def add_student_abs(id_student , id_subject , id_time , arrive_time, duration, is_late, is_abs ):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT * FROM "Students_db".student_abs where id_student  ='{str(id_student) }' and id_subject = '{str(id_subject)}' and id_time = '{str(id_time)}' ;""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = """INSERT INTO "Students_db".student_abs (id_student , id_subject , id_time , arrive_time, duration, is_late, is_abs) VALUES  ('"""+str(id_student)+"""','"""+str(id_subject)+"""','"""+str(id_time)+"""','"""+str(arrive_time)+"""','"""+str(duration)+"""','"""+str(is_late)+"""','"""+str(is_abs)+"""')"""                                      
        val = (subject,id_prof)
        mycursor.execute(sql)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"""SELECT * FROM "Students_db".student_abs where id_student  ='{id_student }' and id_subject = '{id_subject}' and id_time = '{id_time}' ;""")
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [15]:
for m in range (1,6) :
    df = pd.read_csv("/home/ys/notebook/processed/meetings_output/meeting_("+str(m)+")_list_output.csv")
    for index, row in df.iterrows():
        id_student=row['id_student']
        id_subject=row['id_subject']
        id_time=row['id_time']
        arrive_time = row['arrive_time']
        duration = row['duration']
        is_late = row['is_late']
        is_abs = row['is_abs']
        id_student=add_student_abs(id_student , id_subject , id_time , arrive_time, duration, is_late, is_abs )
        print(id_student)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
